In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np

# Зчитування DataFrame з файлу 'drug200.csv' (файл має бути в тій же теці)
df = pd.read_csv('drug200.csv')

print("--- 🔬 Початковий Аналіз Даних ---")
print(df.head())
print(df.info())
print("\nУнікальні значення цільової змінної ('Drug'):")
print(df['Drug'].value_counts())

--- 🔬 Початковий Аналіз Даних ---
   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  DrugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  DrugY
   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  DrugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  DrugY
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5  

In [3]:
# Визначення ознак (X) та цільової змінної (y)
X = df.drop('Drug', axis=1)
y_raw = df['Drug']

# Кодування цільової змінної
le = LabelEncoder()
y = le.fit_transform(y_raw)
class_names = le.classes_ # Зберігаємо імена класів для звіту

# Визначення типів ознак
numerical_features = ['Age', 'Na_to_K']
categorical_features = ['Sex', 'BP', 'Cholesterol']

# Створення препроцесора (ColumnTransformer)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features), # Масштабування числових
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # One-Hot для категоріальних
    ],
    remainder='passthrough'
)

# Розділення на навчальний та тестовий набори
# Використовуємо stratify=y для забезпечення однакового розподілу класів
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nРозмір тренувального набору: {X_train.shape[0]} зразків")
print(f"Розмір тестового набору: {X_test.shape[0]} зразків")


Розмір тренувального набору: 140 зразків
Розмір тестового набору: 60 зразків


In [11]:
from sklearn.model_selection import GridSearchCV

# Створюємо Pipeline з препроцесором та класифікатором (Decision Tree)
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

param_grid = {
    'classifier__max_depth': [3, 5, 7, 10, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(
    model_pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
)

print("\nОптимізація Гіперпараметрів (GridSearchCV)")
grid_search.fit(X_train, y_train)

y_pred_tuned = grid_search.best_estimator_.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)

print(f"Найкращі знайдені параметри: {grid_search.best_params_}")
print(f"Найкраща Точність (Accuracy) на тренувальному наборі (CV): {grid_search.best_score_:.4f}")
print(f"Точність на тестовому наборі з налаштованими параметрами: {accuracy_tuned:.4f}")


Оптимізація Гіперпараметрів (GridSearchCV)
Найкращі знайдені параметри: {'classifier__criterion': 'gini', 'classifier__max_depth': 5, 'classifier__min_samples_split': 2}
Найкраща Точність (Accuracy) на тренувальному наборі (CV): 0.9929
Точність на тестовому наборі з налаштованими параметрами: 0.9833
Найкращі знайдені параметри: {'classifier__criterion': 'gini', 'classifier__max_depth': 5, 'classifier__min_samples_split': 2}
Найкраща Точність (Accuracy) на тренувальному наборі (CV): 0.9929
Точність на тестовому наборі з налаштованими параметрами: 0.9833
